In [ ]:
import pandas as pd
import string

In [ ]:
df_artists = pd.read_csv('artists-data.csv')
indexNames = []
for index, row in df_artists.iterrows():
    if type(row.Genres)!=str:
        indexNames.append(index)
df_artists = df_artists.drop(indexNames)
df_artists.head(10)

In [ ]:
df_lyrics = pd.read_csv('lyrics-data.csv')
df_lyrics = df_lyrics.loc[lambda df: df['language'] == 'en']
df_lyrics.head(10)

In [ ]:
df_merged = pd.merge(df_lyrics, df_artists, how='inner', left_on='ALink', right_on='Link')
df_merged = df_merged.drop(['ALink','Link', "SName", "SLink", "language", "Artist", "Songs", "Popularity"], axis=1)
df_merged.head(10)

In [ ]:
for i in range(len(df_merged)):
    temp = df_merged.iloc[i,0]
    temp = temp.replace("\n", " ")
    temp = temp.replace("\t", " ")
    temp = temp.translate(str.maketrans('', '', string.punctuation))
    temp = temp.lower()
    df_merged.iloc[i,0] = temp
df_merged.head(10)

In [ ]:
df_merged.sample(frac = 1).to_csv('cleaned-data.csv', index = False)
df = pd.read_csv('cleaned-data.csv')
df.head(10)

In [ ]:
s = 0
c = 0

for i in range(len(df_merged)):
    s += len(df_merged.iloc[i,0].split())
    c += 1

s/c